In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import re
import json
import webbrowser
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import GridspecLayout
from markdown_it import MarkdownIt

In [ ]:
%run progressbar.ipynb

In [ ]:
%run sidebar.ipynb

In [ ]:
%run dropdown.ipynb

In [ ]:
def get_css_class(css_file, class_name):
    with open(css_file, 'r') as file:
        css = file.read()
        # Use regex to extract the class
        pattern = r'\.' + re.escape(class_name) + r'\s*\{([^}]*)\}'
        match = re.search(pattern, css, re.DOTALL)
        if match:
            css_content = match.group(1).strip()
            return f'\n<style>\n.{class_name} {{\n{css_content}\n}}\n</style>\n'
        else:
            return None

In [2]:
# Custom CSS to modify the question button design
ques_button_css = get_css_class('style.css', 'ques-button')

In [3]:
# Custom CSS to modify the option button design
opt_button_css = get_css_class('style.css', 'opt-button')

In [5]:
# Custom CSS to modify the header design
header_css = get_css_class('style.css', 'custom-header')

In [6]:
# Custom CSS to modify the footer design
footer_css = get_css_class('style.css', 'custom-footer')

In [7]:
# Create an HTML widget for the background color
background_color_html = widgets.HTML(
    value = get_css_class('style.css', 'custom-vbox')
)

In [8]:
# Loading the header image within an image widget
header_prog1 = widgets.Image(value=open("../03_icons/icon_level-banner_1-0_objective-resources.png", "rb").read(), format='png',
                      layout=widgets.Layout(width='auto', height='70px'))

header_prog2 = widgets.Image(value=open("../03_icons/icon_level-banner_2-1_project-level_study-area_site-selection.png", "rb").read(), format='png',
                      layout=widgets.Layout(width='auto', height='70px'))

header_prog3 = widgets.Image(value=open("../03_icons/icon_level-banner_2-2_project-level_duration-timing.png", "rb").read(), format='png',
                      layout=widgets.Layout(width='auto', height='70px'))

header_prog4 = widgets.Image(value=open("../03_icons/icon_level-banner_4-0_target-species.png", "rb").read(), format='png',
                      layout=widgets.Layout(width='auto', height='70px'))

header_prog5 = widgets.Image(value=open("../03_icons/icon_level-banner_5-1_equipment-deployment.png", "rb").read(), format='png',
                      layout=widgets.Layout(width='auto', height='70px'))

header_prog6 = widgets.Image(value=open("../03_icons/icon_level-banner_6-0_data-analysis.png", "rb").read(), format='png',
                      layout=widgets.Layout(width='auto', height='70px'))

In [9]:
header = header_prog4

In [10]:
# Creating a container for header image
header_container = widgets.VBox([header], 
                    layout=widgets.Layout(width='96%', padding_top= '10px', align_items='flex-start', justify_content='center'))

In [ ]:
notebook_name = "sp_info"
notebook_list_name = "ts_list"
footer_name = "footer_in_prog4"

In [ ]:
footer = main(notebook_name, notebook_list_name, footer_name)

In [13]:
# Creating button
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto', border='1px solid black')) #width='max-content'

In [15]:
# Declaring necessary variables and importing neccesary files for storing user answers and keep track of the visited pages
global stored_choices, path

stored_choices = {}
path = "../temp_storage.json"
back_path = "../backtracking.json"
dropdown_path = "../04_lu_tables/links.csv"

In [16]:
# Importing the json file for storing user answers
with open(path, "r") as f:
    stored_choices = json.load(f)

In [17]:
# Importing the json file for keeping track of visited pages
with open(back_path, 'r') as f:
    back_list = json.load(f)

In [18]:
# Reading the CSV data file into a DataFrame
df = pd.read_csv('../new_format.csv', low_memory=False)

In [ ]:
dp_df = pd.read_csv(dropdown_path, low_memory=False)

In [ ]:
drop_link = dp_df.loc[dp_df['field_code'] == 'sp_info', 'info_url'].iloc[0]

In [19]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list
def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [20]:
# gets the question based on value of other column

ques = df.loc[df['question_code'] == 'sp_info', 'question_text'].iloc[0]
opt = df.loc[df['question_code'] == 'sp_info', 'q_option_label'].iloc[0]
opt_codes = df.loc[df['question_code'] == 'sp_info', 'q_option_code'].iloc[0]

opt = split_and_trim(opt)
opt_codes = split_and_trim(opt_codes)

In [21]:
# Factor widget

question = create_expanded_button(ques, 'success')
question.style.button_color = 'lightblue'
question.style.text_color = 'black'
# Add custom class to the button
question.add_class('ques-button')

option1 = create_expanded_button(opt[0], 'success')
option1.style.button_color = '#DAE8FC'
option1.style.text_color = 'black'
option1.add_class('opt-button')

option2 = create_expanded_button(opt[1], 'success')
option2.style.button_color = '#DAE8FC'
option2.style.text_color = 'black'
option2.add_class('opt-button')

option3 = create_expanded_button(opt[2], 'success')
option3.style.button_color = '#F5F5F5'
option3.style.text_color = 'black'
option3.add_class('opt-button')

# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=ques_button_css)
css_widget2 = widgets.HTML(value=opt_button_css)


factor_grid = GridspecLayout(4, 12, grid_gap='5px', height='550px')
factor_grid[2, 1:6] = question
factor_grid[1, 7:9] = option1
factor_grid[2, 7:9] = option2
factor_grid[3, 7:9] = option3


logical_integer_container = widgets.VBox(children=[factor_grid, css_widget, css_widget2], layout=widgets.Layout(
    align_items='center'  # Center align vertically
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))
# logical_integer_container

In [22]:
button_list = [option1, option2, option3]

In [23]:
selected = False

In [24]:
output = widgets.Output()
def option_1_button_clicked(option1):
    with output:
        clear_output()
        global selected_objective, selected_target_sp, selected_sp_type
        global selected
        
        selected = True
        
        for btn in button_list:
            if btn == option1:
                btn.icon='check'
            else:
                btn.icon=''
        
        selected_option = factor_grid[1, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["sp_info"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        if "objective" in stored_choices.keys():
            selected_objective = stored_choices['objective']
        if "obj_targ_sp" in stored_choices.keys():
            selected_target_sp = stored_choices['obj_targ_sp']
        if "sp_type" in stored_choices.keys():
            selected_sp_type = stored_choices['sp_type']
        
factor_grid[1, 7:9].on_click(option_1_button_clicked)

In [25]:
output = widgets.Output()
def option_2_button_clicked(option2):
    with output:
        clear_output()
        global selected_objective, selected_target_sp, selected_sp_type
        global selected
        
        selected = True
        
        for btn in button_list:
            if btn == option2:
                btn.icon='check'
            else:
                btn.icon=''

        selected_option = factor_grid[2, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["sp_info"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        if "objective" in stored_choices.keys():
            selected_objective = stored_choices['objective']
        if "obj_targ_sp" in stored_choices.keys():
            selected_target_sp = stored_choices['obj_targ_sp']
        if "sp_type" in stored_choices.keys():
            selected_sp_type = stored_choices['sp_type']
        
        
factor_grid[2, 7:9].on_click(option_2_button_clicked)

In [26]:
output = widgets.Output()
def option_3_button_clicked(option3):
    with output:
        clear_output()
        global selected_objective, selected_target_sp, selected_sp_type
        global selected
        
        selected = True
        
        for btn in button_list:
            if btn == option3:
                btn.icon='check'
            else:
                btn.icon=''

        selected_option = factor_grid[3, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["sp_info"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        if "objective" in stored_choices.keys():
            selected_objective = stored_choices['objective']
        if "obj_targ_sp" in stored_choices.keys():
            selected_target_sp = stored_choices['obj_targ_sp']
        if "sp_type" in stored_choices.keys():
            selected_sp_type = stored_choices['sp_type']
        
        
factor_grid[3, 7:9].on_click(option_2_button_clicked)

In [27]:
# button.on_click(show_popup(None, drop_link))
button.on_click(lambda b: show_popup(b, drop_link))

In [28]:
close_button.on_click(close_popup)

In [31]:
# Create a buttons for submitting the question
submit_button = widgets.Button(description="Submit & Next >>", layout=widgets.Layout(border='1px solid black'))

In [32]:
# Create a button for going back to previous question
back_button = widgets.Button(description="<< Back", layout=widgets.Layout(border='1px solid black'))

In [ ]:
# Create the text widget with a warning message
warning_widget = widgets.HTML(
    value="""
    <div style="padding: 5px; background-color: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; border-radius: 5px;">
        <strong>Warning!</strong> Select an option first.
    </div>
    """
)

In [ ]:
error = widgets.HBox(
    [warning_widget],
    layout=widgets.Layout(
        width='auto',  
        height='auto',
        margin='auto',  
        align_items='flex-end',  
        justify_content='flex-end' 
    )
)

In [33]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        js_code = f"window.open('{notebook_filename}', '_self');"
#         link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(Javascript(js_code))

back_button.on_click(on_back_button_clicked)

In [35]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()

        if not selected:
            display(error)
        else:
            next_notebook = []

            condition = df.loc[df['page_name'] == 'sp_type', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']:
                next_notebook.append("sp_type")

            condition = df.loc[df['page_name'] == 'sp_occ_restr', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']:
                next_notebook.append("sp_occ_restr")

            condition = df.loc[df['page_name'] == 'sp_hr_size', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']:
                next_notebook.append("sp_hr_size")

            condition = df.loc[df['page_name'] == 'sp_size', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']:
                next_notebook.append("sp_size")

            condition = df.loc[df['page_name'] == 'sp_rarity', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if (selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']) or (selected_objective in condition_json['objective'] and selected_sp_type in condition_json['sp_type']):
                next_notebook.append("sp_rarity")

            condition = df.loc[df['page_name'] == 'sp_detprob_cat', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']:
                next_notebook.append("sp_detprob_cat")

            condition = df.loc[df['page_name'] == 'cam_independent', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if selected_objective in condition_json['objective']:
                next_notebook.append("cam_independent")

            if next_notebook:
                next_notebook = next_notebook
            else:
                next_notebook = ["test_recommendation-Copy"]


            notebook_filename = next_notebook[0] + ".ipynb"

            # JavaScript to open the URL
            js_code = f"window.open('{notebook_filename}', '_self');"

            back_list.append("sp_info")
            with open (back_path, "w")as f:
                json.dump(back_list, f)

            display(Javascript(js_code))

submit_button.on_click(on_button_clicked)

In [42]:
# Creating containers for all the widgets along with their css properties

all_button = widgets.HBox(
    [back_button, submit_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='space-between' 
    )
)
buttons = widgets.VBox(
    [output, all_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',  
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='center'  
    )
)



upper = widgets.VBox(
    [logical_integer_container, buttons],
    layout=widgets.Layout(
        width='95%',  
        height='auto', 
        border='solid 1px grey',
        padding='10px',
        margin='auto',  
        align_items='center',  
        justify_content='center' 
    )
)
upper.add_class("custom-vbox")

popup = widgets.VBox(children=[button, output_popup], layout=widgets.Layout(
        width='95%',  
        height='auto',  
#         border='solid 2px grey',
        padding='5px',
        margin='auto',  
        align_items='center', 
        justify_content='center'  
    ))


right_col = widgets.VBox(children=[header_container, background_color_html, upper, footer], layout=widgets.Layout(
        width='88%',  
        height='100%',  
        border_left='solid 1px grey',
        padding='5px',
        margin='auto',  
        align_items='center',  
        justify_content='center'
))

cols = widgets.HBox(
    [left_col, right_col],
    layout=widgets.Layout(
        width='95%',  
        height='auto',  
        border='solid 2px grey',
#         padding='5px',
        margin='auto',  
        align_items='flex-start', 
        justify_content='center'
    )
)

display(cols)